### Module 0: Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import pprint
import os 
from time import time 
from dotenv import load_dotenv
import json
import argparse
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from openai import OpenAI
import certifi

# Override bad SSL_CERT_FILE if set
os.environ["SSL_CERT_FILE"] = certifi.where()

from sentence_transformers import SentenceTransformer
import torch
import sklearn

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Importing VE libraries
from prompt_cot import cot_prompt
from utils import *
from dataset_utils import read_wikiqa_data
from prompt_helper import get_joint_prompt_helper, normalize_prediction
from dataset_utils import read_wikiqa_data, wiki_evaluation


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
load_dotenv()
client = OpenAI()

In [5]:
from consistency import _parse_args as consistency_parse_args, in_context_manual_prediction  as consistency_in_context_manual_prediction, post_process_consistency, evaluate_manual_predictions as consistency_evaluate_manual_predictions
from verifying_questions import _parse_args as verifying_questions_args, in_context_manual_prediction as vq_in_context_manual_prediction
from relevant_context import _parse_args as relevant_context_args
from verifying_answers import in_context_manual_prediction as verifying_answers_in_context_manual_prediction, _parse_args as verifying_answers_args
from answer_again import rationale as answer_again_rationale, in_context_manual_prediction as answer_again_in_context_manual_prediction
from evaluation import evaluate_model_performance_v2

### Module 1: Building Consolidated function

In [6]:
# model embedding function required for relevant context
def model_embeddings(sentence, model):
    embedding = model.encode([sentence])
    return embedding[0] #should return an array of shape 384

In [12]:
[i for i in dev_set if i["id"] == id]

[{'id': '8f8aaf0b088711ebbd6dac1f6bf848b6',
  'question': 'Who is younger, Aneta Hladíková or René Poupardin?',
  'answer': 'Aneta Hladíková',
  'all_pars': ['René de Ceriziers( or René de Cerisiers; 1609–1662) was a French theologian and historian.',
   'He was born in Nantes, and entered the Society of Jesus in 1622.',
   'He taught in several Jesuit colleges, and in 1641 left the order.',
   'He became chaplain of the Duke of Orléans and of Louis XIV.',
   'His writings include:',
   'Anton Shunto( born 31 May 1988) is a Belarusian professional footballer.',
   'As of 2019, he plays for Molodechno.',
   'He is a younger brother of Denis Shunto, who is a founder and former president of Krumkachy Minsk.',
   'Markus Weissenberger( born March 8, 1975) is an Austrian former footballer who played as a midfielder.',
   'He is a younger brother of former Austria international player Thomas Weissenberger.',
   'Dmitri Nikolayevich Varfolomeyev( born 15 March 1978 in Leningrad) is a former R

In [12]:
def run_iterative_prediction(id):

    # Consistency Arguments
    args = consistency_parse_args()

    output_file = f"misc/test.jsonl"

    # # Load existing IDs from output file
    # existing_ids = set()
    # if os.path.exists(output_file):
    #     with open(output_file, "r") as f:
    #         for line in f:
    #             try:
    #                 data = json.loads(line.strip())
    #                 existing_ids.add(data["id"])
    #             except:
    #                 continue

    # train_set = read_wikiqa_data("data/train_subset.json", manual_annotation_style=args.style)
    # train_set = train_set[args.train_slice:(args.train_slice + args.num_shot)]

    dev_set = read_wikiqa_data("data/dev_sampled.json")
    # dev_set = dev_set[args.dev_slice:(args.dev_slice + args.num_dev)]
    dev_set = [i for i in dev_set if i["id"] == id]

    results = []
    total_start = time()

    for idx, ex in enumerate(tqdm(dev_set, desc="Processing one-by-one")):
        # if ex['id'] in existing_ids:
        #     continue

        start = time()
        try:
            # Step 1: Predict
            raw_pred = consistency_in_context_manual_prediction(ex, cot_prompt, engine=args.engine, prompt_helper=args.helper, length_test_only=False, n=args.num_shot)
            
            # Step 2: Consistency post-processing
            con_score, final_pred = post_process_consistency(ex, raw_pred, args)
            final_pred["consistency"]= con_score

            # Step  3: check if the consistency is higher than the threshold
            args =verifying_questions_args()

            # if con_score > args.consistency_threshold:
            #     # Step 4: Storing the result if consistency is higher than the threshold 
            #     final_pred['high_consistency'] = True
            #     final_pred["time_taken_sec"] = round(time() - start, 2)
            #     with open(output_file, "a") as fout:
            #         fout.write(json.dumps(final_pred) + "\n")

            # else:
            start2 = time()
            final_pred["consistency_time"] = round(start2 - start, 2)
            final_pred['high_consistency'] = False
            # Step 5: Generating verifying questions
            vq = []
            sentences = rationale_tokenize(final_pred['rationale'])
            for q, s in enumerate(sentences):
                question = vq_in_context_manual_prediction(final_pred['question'], s, args.engine, args.model, args.helper)
                print("Sentence: ", s)
                print("Question", question)
                if question != None:
                    vq.append(question['text'])
                
            #     final_pred["verifying_questions"] = vq

            #     # Step 5: Pulling Relevant Context
            #     start3 = time()
            #     final_pred["time_verifying_answer"] = round(start3 - start2, 2)
            #     args = relevant_context_args()

            #     contexts = []
            #     embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

            #     sentences = rationale_tokenize(final_pred['rationale'])
            #     all_pars_text = []
            #     all_pars = []
            #     va = []
            #     for j, s in enumerate(sentences):
            #         pars_text = get_texts_to_rationale_wikipedia(vq[j], False)
                
            #         pars_text = list(dict.fromkeys(pars_text)) #remove potential duplicates
            #         all_pars_text += pars_text

            #         if pars_text != []:
            #                 sen_embeds = [model_embeddings(s, embedding_model)]
            #                 par_embeds = [model_embeddings(s, embedding_model) for s in pars_text]

            #                 pars = sklearn.metrics.pairwise.pairwise_distances(sen_embeds, par_embeds)
            #                 pars = pars.argsort(axis = 1)[0][:args.topk]

            #                 pars = [pars_text[i] for i in pars]
            #                 contexts.append(pars)
            #                 all_pars += pars
                    
            #         # Step 6: Creating Verified Answers
            #         answer = verifying_answers_in_context_manual_prediction(args.engine, args.helper, args.model, pars, vq[j])
            #         va.append(answer['text'].lstrip())
                
            #     start4 = time()
            #     final_pred["time_context_and_verifying_answer"] = round(start4- start3,2)
            #     final_pred["context"] = contexts
            #     final_pred["verifying_answers"] = va

            #     # Step 7: Answering Again
            #     new_rationale = answer_again_rationale(va)
            #     new_p = answer_again_in_context_manual_prediction(ex, cot_prompt, args.engine, args.helper, args.model,
            #         new_rationale )

            #     start5 =time()
            #     final_pred["time_answer_again"] = round(start5 - start4, 2)
            #     final_pred["new_answer"] = new_p["text"]
                
            #     final_pred["time_taken_sec"] = round(time() - start, 2)
            #     with open(output_file, "a") as fout:
            #         fout.write(json.dumps(final_pred) + "\n")
            
            # results.append(final_pred)

            # # Step 4: Save this prediction to file immediately
            # with open(output_file, "a") as fout:
            #     fout.write(json.dumps(final_pred) + "\n")

        except Exception as e: 
            print(f"Error in record {idx} ({ex['id']}): {e}")
            continue

    total_duration = round(time() - total_start, 2)
    avg_time = total_duration / len(results) if results else 0
    print(f"\n✅ Finished {len(results)} records in {total_duration:.2f} seconds (avg {avg_time:.2f} sec/record)")
    print(f"📄 Results saved to: {output_file}")

    return None

In [13]:
id ="8f8aaf0b088711ebbd6dac1f6bf848b6"

In [14]:
run_iterative_prediction(id)

7 not found


Processing one-by-one:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt for question:Who is younger, Aneta Hladíková or René Poupardin? and Aneta Hladíková was born on 20 August 1993. :
 Write a question that validates the reason for an overall question.

Overall Question: What is the date of death of the composer of film Baalaraajana Kathe?
Reason: First, the composer of film Baalaraajana Kathe is S. Hanumantha Rao.
Question: Who is the composer of film Baalaraajana Kathe?

Overall Question: Who lived longer, Edward Frederick Sanderson or Forrest Towns?
Reason: First, Edward Frederick Sanderson died at age 81.
Question: How long did Edward Frederick Sanderson live for?

Overall Question: Who is younger, Aneta Hladíková or René Poupardin?
Reason: Aneta Hladíková was born on 20 August 1993. 
Question:
Sentence:  Aneta Hladíková was born on 20 August 1993. 
Question {'text': 'When was Aneta Hladíková born?'}
Prompt for question:Who is younger, Aneta Hladíková or René Poupardin? and René Poupardin was born on 21 August 1993.:
 Write a question that val

Processing one-by-one: 100%|██████████| 1/1 [00:04<00:00,  4.86s/it]

Sentence:  René Poupardin was born on 21 August 1993.
Question {'text': 'When was Aneta Hladíková born?'}

✅ Finished 0 records in 4.86 seconds (avg 0.00 sec/record)
📄 Results saved to: misc/test.jsonl


In [15]:
x = ['The Trap is a 1959 American color film noir directed by Norman Panama and released through Paramount Pictures.', 'The Trap may refer to:\n\nFilm\nThe Trap (1913 film), a lost silent film starring Lon Chaney\nThe Trap (1914 film), a dramatic short starring Harry von Meter\nThe Trap (1918 film), an American drama film starring Alice Brady\nThe Trap (1919 film), an American drama film starring Olive Tell\nThe Trap (1922 film), an American silent Western starring Lon Chaney\nThe Trap (1946 film), a Charlie Chan film starring Sidney Toler\nThe Trap (1949 film), an Argentine film directed by Carlos Hugo Christensen\nThe Trap (1950 film), a Czech drama directed by Martin Frič\nThe Trap (1959 film), a crime drama directed by Norman Panama\nThe Trap (1966 film), an adventure/romance starring Oliver Reed and Rita Tushingham\nThe Trap (1985 film), an Italian erotic thriller starring Tony Musante and Laura Antonelli\nJaal: The Trap, a 2003 Indian Hindi-language film starring Sunny Deol and Tabu\nThe Trap (2007 film) or Klopka, a Serbian film\nThe Trap, a 2007 short film starring Jeanne Tripplehorn and Camilla Belle\nFaand: The Trap, a 2014 Bangladeshi film starring Shakib Khan\nThe Trap, a 2017 short film in competition at the 67th Berlin International Film Festival\nThe Trap (2019 film), a film starring Mike Epps and T.I.', 'Chakravyuham: The Trap, a 2023 Indian Telugu-language film\n\nTelevision\nEpisodes\n"The Trap", 12 O\'Clock High season 1, episode 23 (1965)\n"The Trap", 21 Beacon Street episode 10 (1959)\n"The Trap", A Rubovian Legend, series 2, episode 16 (1962)\n"The Trap", Alcoa Presents One Step Beyond season 3, episode 8 (1960)\n"The Trap", Alfred Hitchcock Presents season 10, episode 18 (1965)\n"The Trap", Armored Trooper Votoms episode 5 (1983)\n"The Trap", Bonanza season 6, episode 26 (1965)\n"The Trap", Captain Petko Voivode part 1, episode 2 (1981)\n"The Trap" (Captain Scarlet and the Mysterons), episode 13 (1967)\n"The Trap", Casualty series 23, episode 34 (2009)\n"The Trap", Cheyenne season 2, episode 8 (1956)\n"The Trap", Code 3 episode 17 (1957)\n"The Trap" (Code Lyoko) season 1, episode 14 (2003)\n"The Trap", Code Lyoko: Evolution episode 19 (2020)\n"The Trap", Corky and White Shadow episode 7 (1956)\n"The Trap", Craig Kennedy, Criminologist episode 25 (1953)\n"The Trap", Damaged season 1, episode 12 (2018)\n"The Trap", Danger Man series 1, episode 31 (1961)\n"The Trap", Daniel Boone season 2, episode 26 (1966)\n"The Trap", Diver Dan episode 18 (1960)\n"The Trap", Douglas Fairbanks Presents season 2, episode 6 (1954)\n"The Trap" (Fallout) season 1, episode 6 (2024)\n"The Trap", Flipper & Lopaka season 1, episode 24 (1999)\n"The Trap", Free Meek episode 2 (2019)\n"The Trap", Fright Krewe season 1, episode 9 (2023)\n"The Trap", Have Gun – Will Travel season 5, episode 25 (1962)\n"The Trap", Highway Patrol season 4, episode 14 (1959)\n"The Trap", Hopalong Cassidy season 1, episode 15 (1952)\n"The Trap", Kaatelal & Sons episode 146 (2021)\n"The Trap", Kana Kaanum Kaalangal season 1, episode 21 (2022)\n"The Trap", Land of the Giants episode 8 (1968)\n"The Trap", Les Misérables episode 8 (1967)\n"The Trap", M Squad season 2, episode 5 (1958)\n"The Trap", Maigret (1960 TV series) series 3, episode 12 (1962)\n"The Trap", Man from Interpol episode 11 (1960)\n"The Trap", Planet of the Apes episode 3 (1974)\n"The Trap", Play of the Week season 5, episode 28 (1960)\n"The Trap", P.O.W.']

In [16]:

print(x)

['The Trap is a 1959 American color film noir directed by Norman Panama and released through Paramount Pictures.', 'The Trap may refer to:\n\nFilm\nThe Trap (1913 film), a lost silent film starring Lon Chaney\nThe Trap (1914 film), a dramatic short starring Harry von Meter\nThe Trap (1918 film), an American drama film starring Alice Brady\nThe Trap (1919 film), an American drama film starring Olive Tell\nThe Trap (1922 film), an American silent Western starring Lon Chaney\nThe Trap (1946 film), a Charlie Chan film starring Sidney Toler\nThe Trap (1949 film), an Argentine film directed by Carlos Hugo Christensen\nThe Trap (1950 film), a Czech drama directed by Martin Frič\nThe Trap (1959 film), a crime drama directed by Norman Panama\nThe Trap (1966 film), an adventure/romance starring Oliver Reed and Rita Tushingham\nThe Trap (1985 film), an Italian erotic thriller starring Tony Musante and Laura Antonelli\nJaal: The Trap, a 2003 Indian Hindi-language film starring Sunny Deol and Tabu\

In [17]:
len(x)

3

In [18]:
x[0]

'The Trap is a 1959 American color film noir directed by Norman Panama and released through Paramount Pictures.'

In [19]:
x[1]

'The Trap may refer to:\n\nFilm\nThe Trap (1913 film), a lost silent film starring Lon Chaney\nThe Trap (1914 film), a dramatic short starring Harry von Meter\nThe Trap (1918 film), an American drama film starring Alice Brady\nThe Trap (1919 film), an American drama film starring Olive Tell\nThe Trap (1922 film), an American silent Western starring Lon Chaney\nThe Trap (1946 film), a Charlie Chan film starring Sidney Toler\nThe Trap (1949 film), an Argentine film directed by Carlos Hugo Christensen\nThe Trap (1950 film), a Czech drama directed by Martin Frič\nThe Trap (1959 film), a crime drama directed by Norman Panama\nThe Trap (1966 film), an adventure/romance starring Oliver Reed and Rita Tushingham\nThe Trap (1985 film), an Italian erotic thriller starring Tony Musante and Laura Antonelli\nJaal: The Trap, a 2003 Indian Hindi-language film starring Sunny Deol and Tabu\nThe Trap (2007 film) or Klopka, a Serbian film\nThe Trap, a 2007 short film starring Jeanne Tripplehorn and Camill

In [20]:
x[2]

'Chakravyuham: The Trap, a 2023 Indian Telugu-language film\n\nTelevision\nEpisodes\n"The Trap", 12 O\'Clock High season 1, episode 23 (1965)\n"The Trap", 21 Beacon Street episode 10 (1959)\n"The Trap", A Rubovian Legend, series 2, episode 16 (1962)\n"The Trap", Alcoa Presents One Step Beyond season 3, episode 8 (1960)\n"The Trap", Alfred Hitchcock Presents season 10, episode 18 (1965)\n"The Trap", Armored Trooper Votoms episode 5 (1983)\n"The Trap", Bonanza season 6, episode 26 (1965)\n"The Trap", Captain Petko Voivode part 1, episode 2 (1981)\n"The Trap" (Captain Scarlet and the Mysterons), episode 13 (1967)\n"The Trap", Casualty series 23, episode 34 (2009)\n"The Trap", Cheyenne season 2, episode 8 (1956)\n"The Trap", Code 3 episode 17 (1957)\n"The Trap" (Code Lyoko) season 1, episode 14 (2003)\n"The Trap", Code Lyoko: Evolution episode 19 (2020)\n"The Trap", Corky and White Shadow episode 7 (1956)\n"The Trap", Craig Kennedy, Criminologist episode 25 (1953)\n"The Trap", Damaged seas